In [ ]:
%load_ext aiida
%aiida

import numpy as np
import ase
import urllib.parse as urlparse

import ipywidgets as ipw
from aiida import orm
from aiidalab_widgets_base import viewer

from surfaces_tools import helpers
from surfaces_tools.widgets import comments, obsolete

from aiida_nanotech_empa.utils import cycle_tools as cyc

#import pymol_render.render as pr
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patheffects as path_effects
import matplotlib.font_manager as font_manager
from matplotlib.patches import Polygon, Rectangle

In [ ]:
pk = urlparse.parse_qs(urlparse.urlsplit(jupyter_notebook_url).query)["pk"][0]
workcalc = orm.load_node(pk)

In [ ]:
def get_output_dictionaries(workchain, nm_list, multiplicity_list):
    """
    Retrieve output dictionaries from the workchain based on nm_list and multiplicity_list.

    :param workchain: The workchain node
    :param nm_list: List of (n, m) tuples
    :param multiplicity_list: List of multiplicities
    :return: Dictionary of output dictionaries
    """
    output_dicts = {}
    
    for nm in nm_list:
        for multiplicity in multiplicity_list:
            key = f"cas_{nm[0]}_{nm[1]}_m{multiplicity}_out_params"
            if key in workchain.outputs:
                output_dicts[key] = workchain.outputs[key].get_dict()
    
    return output_dicts

output_dicts = get_output_dictionaries(workcalc, workcalc.inputs.nm_list, workcalc.inputs.multiplicity_list)
# pywidget dropdown  populated with possible combinations from the input multiplicity_list and input nm_list.
cases_widget = ipw.Dropdown(options=list(output_dicts.keys()), description="Outputs:")
output_widget = ipw.HTML()

# function to print on screen the outputs data for the selected dictionary.
def print_dictionary():
    key = cases_widget.value
    output_dict = output_dicts[key]
    output_str=""
    
    if "casscf_energy_ev" in output_dict:
        output_str += f"<pre>CASSCF energy: {output_dict['casscf_energy_ev']} eV</pre>"
    if "casmp2_energy_ev" in output_dict:
        output_str +=  f"<pre>CASMP2 energy: {output_dict['casmp2_energy_ev']} eV</pre>"
    output_widget.value = output_str

cases_widget.observe(lambda c: print_dictionary(), names='value')
display(ipw.VBox([cases_widget,output_widget]))

# Comments

In [ ]:
comments_widget = comments.CommentsWidget(workchain=pk)
display(comments_widget)

# Mark calculation as obsolete 

In [ ]:
obsolete = obsolete.ObsoleteWidget(workchain=pk)
display(obsolete)